# Monte Carlo Localization
Monte Carlo Localization (MCL)とはパーティクルと呼ばれる空間内のロボットの状態を表現する粒子を用いて,確率分布によってロボットの自己位置推定を行う手法である.このノートブックではMCLの実行に必要最低限のロボットの状態遷移モデル（対向２輪型ロボット）や点ランドマークの観測方程式,パーティクルの状態遷移モデル,パーティクルの尤度計算,リサンプリング等の説明とPythonでの実装で構成されている.   
本ノートブックはMCLをフルスクラッチでコーディングし,自身の学習,講義内容の理解を深めるためのものである.    
(上田隆一先生の書籍,詳解確率ロボティクスを参考にして,MCLの実装を行いました.)

### ヘッダーのインポート、シミュレーション時間の設定

In [17]:
%matplotlib nbagg

import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from scipy.stats import norm

animation_play_interval = 100

dt = 0.05                       #サンプリングタイム
simu_time = 10                  #シミュレーション時間
simu_frames = int(simu_time/dt) #シミュレーションのフレーム数

## 対向２輪型ロボットの初期値設定と状態遷移関数の定義
今回使用するロボットは対向２輪型ロボットである.対向２輪型ロボットは２つの車輪が機体を挟んで対向に配置されたロボットのことである.このロボットをモデリングする際,タイヤの直径や間隔、車輪のモーターによる推進トルクやロボットの加速度なども考慮しなければならない.しかし,本ノートブックでは簡単にロボットの速度と角速度を設定するとその通りに動くと仮定し実装を行う.  
### ロボットに入力する指令値,ロボットの初期値を定義
下図は対向２輪型ロボットを記号化して表現している.  
ロボット初期姿勢：(x,y,theta) = (0[m],0[m],0[rad])    
速度指令値  　  ：ν = 1.0[m/s]  
角速度指令値    ：ω = 0.5[rad/s]  
ロボットの半径  ：0.2[m]
<img src="images/robot_model2.png" alt="aaaa" width="500">

In [18]:
pose  = np.array([0,0,0])  #ロボットの状態を定義 ([x,y,theta])
ref = np.array([1.0,-0.5]) #ロボットに対する入力を定義 ([速度, 角速度])
robot_r = 0.2

<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
</script>
<script type="text/x-mathjax-config">
 MathJax.Hub.Config({
 tex2jax: {
 inlineMath: [['$', '$'] ],
 displayMath: [ ['$$','$$'], ["\\[","\\]"] ]
 }
 });
</script>

### ロボットの状態遷移関数
$ \Delta t $をサンプリング時間,$ t $を時刻,$ t-1 $を１ステップ前の時刻とするとロボットの状態遷移関数は以下のように与えられる.

$$ 
\left(
\begin{matrix}
x_t \\
y_t \\
\theta_t 
\end{matrix}
\right)
+
\left(
\begin{matrix}
x_{t-1} \\
y_{t-1} \\
\theta_{t-1}
\end{matrix}
\right)
+
\left\{
\begin{array}{ll}
\left(
\begin{matrix}
\nu_t \cos \theta_{t-1} \\
\nu_t \sin \theta_{t-1} \\
\omega_t
\end{matrix}
\right)
\Delta t
& 
\left( 
\omega_t = 0
\right)
\\
\left(
\begin{matrix}
\nu_t \omega_{t}^{-1} \{ \sin(\theta_{t-1} + \omega_t \Delta t) - \sin \theta_{t-1} \} \\
\nu_t \omega_{t}^{-1} \{ -\cos(\theta_{t-1} + \omega_t \Delta t) + \cos \theta_{t-1} \} \\
\omega_t \Delta t \\
\end{matrix}
\right)
&
\left( 
\omega_t \neq 0
\right)
\end{array}
\right.
$$

また,ロボットの角速度指令値$ \omega $が0に近い時とそうでない時で場合分けしている.
この状態遷移関数を次のセルの10~17行目に実装した.
この関数`robot_update(ref,t,pose)`はロボットへの入力`ref(numpy.ndarray 1×2)`と時刻`t`,ロボットの姿勢`pose(numpy.ndarray 1×3)`を引数にとり,ロボットの時刻tにおけるロボットの姿勢`(numpy.ndarray 1×3)`を戻り値としている.

In [19]:
def robot_update(ref,t,pose):
    # ref : ロボットへの入力
    # t : 時刻
    # pose : ロボットの姿勢
    
    v = ref[0]     #ロボットに入力する速度
    omega = ref[1] #ロボットに入力する角速度
    
    x = pose[0]    #ロボットの姿勢
    y = pose[1]
    theta = pose[2]

    if np.fabs(omega) < 1.0e-10:
        nx = x + v*np.cos(theta)*t
        ny = y + v*np.sin(theta)*t
        ntheta = theta + omega*t
    else:    
        nx = x + (v/omega)*( np.sin(theta+omega*t)-np.sin(theta))
        ny = y + (v/omega)*(-np.cos(theta+omega*t)+np.cos(theta))
        ntheta = theta + omega*t

    npose = np.array([nx,ny,ntheta])
    return npose

### ランドマークの定義
ランドマークの位置の定義を次のセルで行っている.
今回例として$ (x,y) = (2,2), (0,4), (-3,-3) $の３か所に点ランドマークを設置した.    
また,ロボットが点ランドマークを観測する際のノイズを距離と角度それぞれ`dis_std`と`ang_std`で定義している.

In [20]:
lm = np.array([[2,2],[0,4],[-3,-3]])  #ランドマークの位置を定義
lm_num = lm.shape[0]                  #ランドマークの個数
dis_std = 0.1 #距離の観測ノイズの標準偏差
ang_std = 0.1 #角度の観測ノイズの標準偏差

<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
</script>
<script type="text/x-mathjax-config">
 MathJax.Hub.Config({
 tex2jax: {
 inlineMath: [['$', '$'] ],
 displayMath: [ ['$$','$$'], ["\\[","\\]"] ]
 }
 });
</script>

### 点ランドマークの観測
ロボットのセンサから点ランドマークを観測した場合,下図のようにセンサの出力としてランドマークとの距離$ L_j $とロボットから見たランドマークの向き$ \phi_j $を得られるとする.ロボットの姿勢(x,y,θ)とランドマークの位置から観測方程式を立てると以下で表せる.  
$ L_j $はロボットとランドマーク間の距離であるから,  

$$ 
L_j = \sqrt{ \left( m_{j,x}-x \right)^2 + \left( m_{j,y}-y \right)^2}
$$
$ \phi_j $はロボットから見た時のランドマークの観測角度なので,
$$
\phi_j = \arctan2(m_{j,y}-y,m_{j_x}-x)-\theta
$$
これらをまとめると,観測方程式となる
$$
\left(
\begin{matrix}
L_j \\
\phi_j
\end{matrix}
\right)
=
\left(
\begin{matrix}
\sqrt{ \left( m_{j,x}-x \right)^2 + \left( m_{j,y}-y \right)^2} \\
\arctan2(m_{j,y}-y,m_{j_x}-x)-\theta
\end{matrix}
\right)
$$
この観測方程式は次のセルの13～14行目,26～27行目に実装されている.
<img src="images/robot_observation2.png" width="500">

<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
</script>
<script type="text/x-mathjax-config">
 MathJax.Hub.Config({
 tex2jax: {
 inlineMath: [['$', '$'] ],
 displayMath: [ ['$$','$$'], ["\\[","\\]"] ]
 }
 });
</script>

### パーティクルの尤度計算
実ロボットの点ランドマークの観測をパーティクルに当てはめた時,そのパーティクルがロボットの姿勢としてどの程度尤もらしいか計算を行っている.
パーティクルの姿勢の尤度計算は次のセルの30～31行目に実装されている.

<img src="images/observation_and_likelihood.png" width="450">

### 各ロボットのパーティクルの尤度を計算する関数
この関数`observation_likelihood`は引数としてロボットの姿勢`robot_pose (numpy.ndarray 1×3)`,パーティクルの姿勢`particle_pose (numpy.ndarray 1×3)`,ランドマークの位置`landmark (numpy.ndarray)`,距離と角度の観測値のノイズの標準偏差をとり,戻り値はパーティクルの尤度である

In [21]:
def observation_likelihood(robot_pose, particle_pose, landmark, dis_std, ang_std):
    # robot_pose ： ロボットの姿勢
    # particle_pose :　パーティクルの姿勢
    # landmark ; ランドマークの位置
    # dis_std : ランドマークとの観測距離に付与されるノイズの標準偏差
    # ang_std : ランドマークとの観測角度に付与されるノイズの標準偏差
    
    observation_likelihood = 1.0 #尤度の初期化
    
    #ロボットの状態を各変数に代入
    robot_x = robot_pose[0]
    robot_y = robot_pose[1]
    robot_theta = robot_pose[2]
    
    # ロボットから見たランドマークの観測値を計算
    obs = np.zeros((lm_num,2)) #ランドマークの個数分の観測定義：ランドマークの個数×２（距離、角度）の行列の定義
    
    for j in range(lm_num):
        obs[j][0] = np.sqrt((landmark[j][0]-robot_x)**2 + (landmark[j][1]-robot_y)**2)
        obs[j][1] = np.arctan2(landmark[j][1]-robot_y,landmark[j][0]-robot_x) - robot_theta
        
        # 距離の観測値に対して正規分布に従うノイズの付与
        dis_noise = np.random.normal(0, np.abs(dis_std))
        obs[j][0] += dis_noise
        # 角度の観測値に対して正規分布に従うノイズの付与
        ang_noise = np.random.normal(0, ang_std)
        obs[j][1] += ang_noise
        
    
    # 各パーティクルの観測値を計算し、ガウス分布から尤度を計算
    for i in range(lm_num):
        p_distance = np.sqrt((landmark[i][0] - particle_pose[0])**2 + (landmark[i][1] - particle_pose[1])**2)
        p_angle = np.arctan2(landmark[i][1] - particle_pose[1], landmark[i][0] - particle_pose[0]) - particle_pose[2]

        #距離、角度の尤度を計算
        likelihood_distance = (1.0 / (np.sqrt(2 * np.pi) * dis_std)) * np.exp(-(1/2) * ((obs[i][0] - p_distance) / dis_std)**2)
        likelihood_angle = (1.0 / (np.sqrt(2 * np.pi) * ang_std)) * np.exp(-(1/2) * ((obs[i][1] - p_angle) / ang_std)**2)

        observation_likelihood *= likelihood_distance * likelihood_angle

    return observation_likelihood


<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
</script>
<script type="text/x-mathjax-config">
 MathJax.Hub.Config({
 tex2jax: {
 inlineMath: [['$', '$'] ],
 displayMath: [ ['$$','$$'], ["\\[","\\]"] ]
 }
 });
</script>

### パーティクルの設定,パーティクルの更新,入力の誤差
次のセルでパーティクルの個数やパーティクルの初期位置の設定を行っている.また,パーティクルの初期位置はロボットの初期位置を既知として同じ場所に置いている.  
パーティクルの姿勢は対向２輪型ロボットの状態遷移モデルに沿って更新されるが,実際にパーティクルに入力する速度と角速度はロボットへの入力にノイズを付与したものである.ロボットが動いたときに生じる雑音の標準偏差をそれぞれ$ \left( \sigma_{\nu \nu},\sigma_{\nu \omega},\sigma_{\omega \nu},\sigma_{\omega \omega} \right) $とすると,パーティクルの入力にノイズを付与した$ \left( \nu^\prime,\omega^\prime \right) $は次のように表せる.

$$
\left(
\begin{matrix}
\nu^\prime \\
\omega^\prime
\end{matrix}
\right)
=
\left(
\begin{matrix}
\nu \\
\omega
\end{matrix}
\right)
+
\left(
\begin{matrix}
\sigma_{\nu \nu} \sqrt{|\nu|/\Delta t} + \sigma_{\nu \omega} \sqrt{|\omega|/\Delta t} \\
\sigma_{\omega \nu} \sqrt{|\nu|/\Delta t} + \sigma_{\omega \omega} \sqrt{|\omega|/\Delta t}
\end{matrix}
\right)
$$
この入力にノイズを付与する関数`noise_ref(ref)`は次のセルの13～23行目に実装されている.  
パーティクルの姿勢の更新を行う関数`particle_update(ref,t,pose)`は次のセル26～45行目に実装されている.

In [22]:
particle_num = 20   #パーティクルの個数を定義

p = np.zeros((particle_num,3)) #パーティクルの初期位置を定義（ロボットの初期位置と同じ）
pw = np.zeros(particle_num)    #パーティクルの重みを格納する配列の定義

nn = 0.2#ロボットが1[m]進んだ時の距離のばらつきの標準偏差
no = 0.2#ロボットが1[rad]回転した時の距離のばらつきの標準偏差
on = 0.2#ロボットが1[m]進んだ時の向きのばらつきの標準偏差
oo = 0  #ロボットが1[rad]回転した時の向きのばらつきの標準偏差
input_std_dev = np.array([nn,no,on,oo]) #標準偏差を行列にまとめる

#入力にノイズを付与する関数
def noise_ref(ref):
    # ref : 入力（速度、角速度）
    noise_ref = np.zeros(2)
    rng = np.random.default_rng()
    
    noise = np.zeros(4)
    for i in range(4):
        noise[i] = rng.normal(0,input_std_dev[i]) #正規分布に従う乱数の生成
    
    noise_ref[0] = ref[0] + (noise[0]*np.sqrt(np.abs(ref[0])/dt) + noise[1]*np.sqrt(np.abs(ref[1])/dt))
    noise_ref[1] = ref[1] + (noise[2]*np.sqrt(np.abs(ref[0])/dt) + noise[3]*np.sqrt(np.abs(ref[1])/dt))
    return noise_ref

#ロボットモデルによるパーティクルの更新
def particle_update(ref,t,pose):
    # ref : 入力 (速度、角速度)
    # t : 時刻[s]
    # pose : パーティクルの姿勢
    ref = noise_ref(ref) #パーティクルへの入力にノイズを付与
    v = ref[0]
    omega = ref[1]
    
    x = pose[0] 
    y = pose[1]
    theta = pose[2]

    if np.fabs(omega) < 1.0e-10:
        nx = x + v*np.cos(theta)*t
        ny = y + v*np.sin(theta)*t
        ntheta = theta + omega*t
    else:    
        nx = x + (v/omega)*( np.sin(theta+omega*t)-np.sin(theta))
        ny = y + (v/omega)*(-np.cos(theta+omega*t)+np.cos(theta))
        ntheta = theta + omega*t

    npose = np.array([nx,ny,ntheta])
    return npose

### パーティクルのリサンプリング
パーティクルの重みに基づいてリサンプリングを行う.重みの高いパーティクルが多く複製され,重みの低いパーティクルが少なくなる.
<img src="images/resampling_particle2.png" width=250><img src="images/resampled_particle2.png" width=250>
  
この関数`resample(particle,particle_weight)`はパーティクルの配列とパーティクルの重み配列を引数にとり,リサンプルされた配列を戻り値としている.

In [23]:
def resample(particle, particle_weight):
    idx = np.arange(len(particle))
    resampled_idx = np.random.choice(idx, size=len(particle), replace=True, p=particle_weight)
    resample_particle = particle[resampled_idx]
    return resample_particle

### MCLの実装
MCLをこれまで作成した関数を使用して18～34行目に実装した.
MCLの手順を次に示す.
1. パーティクルの初期化(今回はパーティクルの個数は２０個,パーティクルの初期姿勢はロボットの初期姿勢と同じ)
2. ロボット姿勢を状態遷移モデルに沿って更新
3. センサでランドマークの観測を行う
4. 観測から各パーティクルがどれだけロボットの姿勢として尤もらしいか計算
5. 各パーティクルの重みを正規化し,重みに基づいて新たなパーティクルを生成
6. 上記を繰り返す

In [24]:
#マップの作成
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal')
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)

loop_cnt = 0 #関数updateをコールバックされた回数
def update(f):
    global ref,dt,pose,lm,lm_num,robot_r,particle_num,p,pw,dis_std,ang_std,loop_cnt
    
    ax.cla() #アニメーションのためにaxをクリア
    ax.set_aspect("equal")#axを再設定
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    ax.grid()
    
    ######################################################
    ########## モンテカルロ・ローカリゼーション ##########
    ######################################################
    
    #ロボットの更新
    pose=robot_update(ref,dt,pose)
    
    #パーティクルの更新
    for n in range(particle_num):
        p[n,:] = particle_update(ref,dt,p[n,:]) #パーティクルのロボットのモデルによる状態遷移
        pw[n] = observation_likelihood(pose,p[n,:],lm,dis_std,ang_std) #観測の更新を行い、各パーティクルの尤度を計算
    
    #リサンプリング（今回は30フレーム間隔で行う）
    if loop_cnt%30 == 1:
        pw = pw / np.sum(pw) #各パーティクルの重みを正規化
        p = resample(p,pw)
        pw[:] = 1 / particle_num 

    ######################################################
    ##########       アニメーションの描画       ##########
    ######################################################
    
    #パーティクルの描画
    for m in range(particle_num):
        ax.plot(p[m][0],p[m][1],marker='.',c='b',alpha=0.3)
        
    #ランドマークの描画
    for r in range(lm_num):
        ax.plot(lm[r][0],lm[r][1],"x",c="green") #Rand mark draw
        ax.plot([pose[0],lm[r][0]],[pose[1],lm[r][1]],c="green") #ロボットとランドマーク間に線分を描画
    
    #ロボットの描画
    c = patches.Circle(xy=(pose[0],pose[1]), radius=robot_r, fc='w', ec='r') #ロボットの胴体を描画
    ax.add_patch(c)
    ax.plot([pose[0],pose[0]+robot_r*np.cos(pose[2])],[pose[1],pose[1]+robot_r*np.sin(pose[2])],c="red") #ロボットの先端を描画
    
    loop_cnt+=1
    
#1フレームごとに関数updateをコールバック、ロボットやパーティクルの更新を行い座標系に描画
anim = FuncAnimation(fig, update, frames=np.arange(0,simu_frames), interval=animation_play_interval)
plt.show() #アニメーションのプロット

<IPython.core.display.Javascript object>

<img src="images/exeimg.png" width=450>